In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [2]:
URL = 'https://www.nespresso.com/uk/en/order/capsules/vertuo'
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

res = requests.get(URL, headers=HEADERS, timeout=5)
# soup = BeautifulSoup(res.content, 'html.parser')
soup = BeautifulSoup(res.content, 'lxml')

In [3]:
names = soup.find_all('script')
# FIXME, I had to find this manually by hand. Must be a better way
NUMBER = 47
#print(names[NUMBER].string)

json_string = names[NUMBER].string
json_string = json_string[json_string.find('{'):json_string.rfind("}")+1]

json_data = json.loads(json_string)

In [5]:
with open('../data/raw_main_data.json', 'w') as outfile:
    json.dump(json_data, outfile)

with open('../data/formatted_raw_main_data.json', 'w') as outfile:
    json.dump(json_data['configuration']['eCommerceData'], outfile)

# Obtaining the individual pod data

In [30]:
import pandas as pd
import pprint
# pprint.pprint(json_data['configuration']['eCommerceData']['products'])

BASE_URL = 'https://www.nespresso.com/'
cols = ['name', 'url', 'intensity', 'price', 'description', 'ranges', 'flavors']
df = pd.json_normalize(json_data['configuration']['eCommerceData']['products'])

# fix urls
df['url'] = BASE_URL + df['url']

# save total data
df.to_csv('../data/summary_coffee_data.csv')

df = df[cols]
df.head()

,name,url,intensity,price,description,ranges,flavors
0,Mild Coffee 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,73.3,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[]
1,Decaf and Half Caff 90 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,50.5,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[]
2,Summer Iced 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,74.2,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[]
3,Single Origin Pack,https://www.nespresso.com//uk/en/order/capsule...,0,63.5,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[]
4,Wellness Boost Pack,https://www.nespresso.com//uk/en/order/capsule...,0,59.4,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[]


# Get descriptions from each pod

In [80]:
def get_description_from_url(url, description_number=9):
    try:
        res = requests.get(url, headers=HEADERS, timeout=5)
        soup = BeautifulSoup(res.content, 'lxml')

        # convert to json
        data = soup.find_all('script')[description_number].string
        if data is None:
            return None
        json_string = data[data.find('{'):data.rfind('}')+1]
        json_data = json.loads(json_string)

        # there is probably a better way to do this
        description = json_data['components'][1]['variations'][0]['properties']['copywriting']['description']

        return description
    except:
        return None

In [81]:
df['description_scraped'] = df['url'].apply(lambda x: get_description_from_url(x))

In [83]:
df.to_csv('../data/raw_coffee_data.csv')

In [84]:
df

,name,url,intensity,price,description,ranges,flavors,description_scraped
0,Mild Coffee 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,73.30,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
1,Decaf and Half Caff 90 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,50.50,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
2,Summer Iced 120 Capsule Assortment,https://www.nespresso.com//uk/en/order/capsule...,0,74.20,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
3,Single Origin Pack,https://www.nespresso.com//uk/en/order/capsule...,0,63.50,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
4,Wellness Boost Pack,https://www.nespresso.com//uk/en/order/capsule...,0,59.40,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
5,MONTHLY FAVOURITES LONG CUP 120 ASSORTMENT,https://www.nespresso.com//uk/en/order/capsule...,0,80.40,None,[nesclub2.uk.b2c/cat/capsule-range-coffee-asso...,[],None
6,Exotic Liminha Over Ice,https://www.nespresso.com//uk/en/order/capsule...,0,0.58,None,"[nesclub2.uk.b2c/cat/capsule-cold-vertuo, nesc...",[],<p>Very refreshing coffee with mint and citrus...
7,Tropical Coconut Over Ice,https://www.nespresso.com//uk/en/order/capsule...,0,0.69,None,"[nesclub2.uk.b2c/cat/capsule-cold-vertuo, nesc...",[nesclub2.uk.b2c/cat/capsule-aromatic-coconut],The coconut flavour enhances the caramel notes...
8,Ice Forte,https://www.nespresso.com//uk/en/order/capsule...,0,0.67,None,"[nesclub2.uk.b2c/cat/capsule-cold-vertuo, nesc...",[],Barista Creations Ice Forte carries that disti...
9,Ice Leggero,https://www.nespresso.com//uk/en/order/capsule...,0,0.56,None,"[nesclub2.uk.b2c/cat/capsule-cold-vertuo, nesc...",[],Chill out with this refreshing iced coffee exp...
